# Optimize Ramp Settings

This notebook outlines an example to optimize the ramp settings for a few different types of observations.

In these types of optimizations, we must consider observations constraints such as saturation levels, SNR requirements, and limits on acquisition time.

**Note**: The reported acquisition time does not include obsevatory and instrument-level overheads, such as slew times, filter changes, script compilations, etc. It only includes detector readout times (including reset frames).

In [2]:
## Standard imports

# Makes print and division act like Python 3
from __future__ import print_function, division

# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Enable inline plotting at lower left
%matplotlib inline
matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'none'

# seaborn package for making pretty plots, but not necessary
try:
    import seaborn as sns
    params =   {'xtick.direction': 'in', 'ytick.direction': 'in', 'font.family': ['serif'],
                'text.usetex': True, 'text.latex.preamble': ['\usepackage{gensymb}']}
    sns.set_style("ticks", params)
except ImportError:
    print('Seaborn module is not installed. Not strictly required, but is a useful package!')
    
from IPython.display import display, Latex, clear_output

In [24]:
import pynrc
from pynrc import nrc_utils
from pynrc.nrc_utils import (webbpsf, poppy, pix_noise, S)
from pynrc.pynrc_core import table_filter

pynrc.setup_logging('WARNING', verbose=False)

from astropy.table import Table

## Example 1: M-Dwarf companion

We want to observe an M-Dwarf companion (K=18) orbiting an F0V primary (K=10 mags). Assume the M-Dwarf flux is not significantly impacted by the primary (in the background limited regime).

We will try a couple different types of observations (direct imaging and coronagraphy).

In [17]:
# Get stellar spectra
bp_k = S.ObsBandpass('k')
sp_M2V = pynrc.stellar_spectrum('M2V', 18, 'vegamag', bp_k)
sp_F0V = pynrc.stellar_spectrum('F0V', 10, 'vegamag', bp_k)

In [41]:
# Initiate a NIRCam observation
nrc = pynrc.NIRCam('F430M', wind_mode='WINDOW', xpix=160, ypix=160)

# Let's assume we want good photometry on the primary to calibrate the M-Dwarf
#  - Set well_frac_max=0.75
# Want a SNR~100 in the F430M filter
#  - Set snr_min=100
t = nrc.ramp_optimize(sp_M2V, sp_bright=sp_F0V, snr_min=100, well_frac_max=0.75)

# Take the Top 2 settings for each readout pattern
t = table_filter(t, 2)
print(t)

 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
RAPID         9  510      2.51   1278.96   1421.06    100.0    0.707    2.651
RAPID         9  511      2.51   1281.47   1423.85    100.1    0.707    2.651
BRIGHT1       5  700      2.51   1755.43   1950.48     99.9    0.707    2.262
BRIGHT1       5  701      2.51   1757.94   1953.27    100.0    0.707    2.262
BRIGHT2       4  751      2.23   1674.07   1883.33    100.0    0.628    2.303
BRIGHT2       4  752      2.23   1676.30   1885.84    100.0    0.628    2.303
SHALLOW4      2  621      2.51   1557.32   1730.35    100.0    0.707    2.403
SHALLOW4      2  622      2.51   1559.83   1733.14    100.1    0.707    2.403


In [46]:
# Do the same thing, but for coronagraphic mask instead
nrc._mask = 'MASK430R'; nrc._pupil = 'CIRCLYOT'
nrc.update_psf_coeff()
nrc.update_detectors(xpix=320, ypix=320)

# We assume that longer ramps will give us the best SNR for time
patterns = ['MEDIUM8', 'DEEP8']
t = nrc.ramp_optimize(sp_M2V, sp_bright=sp_F0V, snr_min=100, well_frac_max=0.75, patterns=patterns)

# Take the Top 2 settings for each readout pattern
t = table_filter(t, 2)
print(t)

 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
MEDIUM8      10   83    104.77   8695.57   8784.30    100.0    0.001    1.066
MEDIUM8      10   84    104.77   8800.34   8890.14    100.6    0.001    1.066
DEEP8        20   12    414.79   4977.45   4990.28     99.0    0.003    1.400
DEEP8        20   13    414.79   5392.24   5406.14    103.0    0.003    1.400


**RESULTS**: Based on these two comparisons, it looks like direct imaging is much more efficient in getting to the requisite SNR. In addition, direct imaging gives us a photometric comparison source that the coronagraphic mask does not.

In [5]:
t = nrc.ramp_optimize(sp_M2V, tacq_max=1000, verbose=True, return_full_table=True, ng_min=5, snr_frac=0, \
                     well_frac_max=0.5)

BRIGHT1
BRIGHT2
DEEP2
DEEP8
MEDIUM2
MEDIUM8
RAPID
SHALLOW2
SHALLOW4
Top 10 results sorted by 'efficiency' (SNR/t_acq):
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
MEDIUM2      10   32     31.34   1002.76   1013.66   2501.7    0.465   78.574
MEDIUM2       9   35     27.93    977.55    989.47   2465.9    0.415   78.390
DEEP2         5   36     27.93   1005.48   1017.74   2497.9    0.415   78.298
MEDIUM2       8   41     24.52   1005.48   1019.45   2495.2    0.364   78.148
MEDIUM2       7   47     21.12    992.54   1008.55   2471.3    0.313   77.817
MEDIUM2       6   57     17.71   1009.57   1028.98   2481.1    0.263   77.347
SHALLOW2     10   63     16.01   1008.55   1030.00   2477.4    0.238   77.191
SHALLOW2      9   71     14.31   1015.70   1039.88   2477.1    0.212   76.815
MEDIUM2       5   71     14.31   1015.70   1039.88   2471.3    0.212   76.636
SHALLOW2      8   81   

In [12]:
print(t[t['Pattern']=='MEDIUM2'])
print(t[t['Pattern']=='DEEP2'])

 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
MEDIUM2       5   71     14.31   1015.70   1039.88   2435.7    0.212   75.532
MEDIUM2       6   57     17.71   1009.57   1028.98   2422.1    0.263   75.507
MEDIUM2       7   49     21.12   1034.77   1051.46   2444.7    0.313   75.392
MEDIUM2       8   42     24.52   1030.00   1044.31   2431.8    0.364   75.250
MEDIUM2       9   37     27.93   1033.41   1046.01   2429.1    0.415   75.106
MEDIUM2      10   33     31.34   1034.09   1045.33   2423.9    0.465   74.968
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
DEEP2         5   35     27.93    977.55    989.47   2421.3    0.415   76.975
DEEP2         6   29     34.74   1007.52   1017.40   2444.3    0.516   76.631
DEEP2         7   25     41.55   1038.86   1047.38   2469.1    0